# **Random Forest**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 6 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.metrics import classification_report, roc_auc_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
from scipy.signal import butter, filtfilt
import random

In [327]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/Annotated/Regular/AveragePooled/split_features_1s_all_1D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [328]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [329]:
train_data

{'melspectrogram': array([[-0.01898465, -0.2811592 , -0.25845313, ..., -0.9981291 ,
         -0.9983617 , -1.        ],
        [-0.01670736, -0.27276382, -0.2651618 , ..., -0.9903572 ,
         -0.9974626 , -1.        ],
        [-0.01776626, -0.26800779, -0.26286083, ..., -0.9897578 ,
         -0.9985065 , -0.9999572 ],
        ...,
        [-0.07289265, -0.5997221 , -0.6325296 , ..., -0.9177227 ,
         -0.93674475, -0.99452364],
        [-0.01720924, -0.9334746 , -0.9393428 , ..., -0.8331133 ,
         -0.8683263 , -0.98977023],
        [-0.06566753, -0.6877171 , -0.70655054, ..., -0.8833477 ,
         -0.89263326, -0.9825436 ]], dtype=float32),
 'mfcc': array([[-9.9767429e-01,  5.7248688e-01, -2.8398493e-01, ...,
          1.3116055e-02,  6.1448671e-02,  3.6781926e-02],
        [-9.9636453e-01,  5.7445627e-01, -2.7706087e-01, ...,
          2.2723455e-02,  6.6032313e-02,  2.8396677e-02],
        [-1.0000000e+00,  5.6324875e-01, -2.5171626e-01, ...,
          3.5349183e-02,  6.71

In [330]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [331]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [332]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [333]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [334]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [335]:
display(train_y.shape)
display(train_y[:15])

(12565,)

array([ 0, 19,  2,  1, 11, 17, 15,  2,  2,  2,  0,  1,  1, 12,  0],
      dtype=int32)

In [336]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(12565, 128)

array([-0.01743852, -0.6508722 , -0.6916726 , -0.63577497, -0.6325771 ,
       -0.693267  , -0.69924164, -0.7079409 , -0.73118097, -0.7504752 ,
       -0.78667325, -0.7978573 , -0.80476624, -0.8203451 , -0.7949259 ,
       -0.789861  , -0.8088392 , -0.82532144, -0.83248717, -0.80361825,
       -0.83153164, -0.85650146, -0.85409397, -0.85105276, -0.8633566 ,
       -0.8759206 , -0.8782255 , -0.8650762 , -0.8831901 , -0.88035285,
       -0.8797179 , -0.8726353 , -0.870042  , -0.86516607, -0.84496367,
       -0.82393944, -0.798188  , -0.7757094 , -0.76359135, -0.7636499 ,
       -0.7554068 , -0.7708649 , -0.7789858 , -0.7851823 , -0.8036501 ,
       -0.82371676, -0.8172532 , -0.7944738 , -0.7687108 , -0.7613045 ,
       -0.73363197, -0.7210863 , -0.73323107, -0.7296212 , -0.7292174 ,
       -0.73931384, -0.76318365, -0.7856355 , -0.7871114 , -0.7552131 ,
       -0.7135708 , -0.67738   , -0.6638213 , -0.6613151 , -0.6510076 ,
       -0.67116624, -0.7190023 , -0.7187137 , -0.65514815, -0.60

'mfcc'

(12565, 20)

...

(12565, 84)

array([-29.39905 , -28.323952, -30.232784, -29.47406 , -27.153494,
       -24.825844, -26.720036, -26.476421, -28.381443, -35.24852 ,
       -35.75719 , -36.161526, -38.84649 , -38.180325, -36.2371  ,
       -36.643337, -38.62381 , -40.752415, -39.763824, -33.694458,
       -34.624397, -36.676933, -29.563038, -31.875635, -36.143166,
       -39.61375 , -39.852917, -40.979412, -41.2093  , -41.830776,
       -41.238277, -40.43606 , -42.073097, -43.597797, -43.47115 ,
       -44.162884, -46.430286, -48.147423, -48.957855, -49.706734,
       -50.141304, -51.65557 , -51.007893, -49.478077, -49.370472,
       -50.613445, -52.21544 , -52.10996 , -50.735115, -53.456318,
       -54.24676 , -55.094383, -56.596664, -56.408787, -55.866055,
       -57.131966, -56.22503 , -54.807167, -53.670033, -52.506256,
       -51.82573 , -51.714436, -53.293312, -53.486908, -52.728363,
       -51.071804, -51.70296 , -52.312763, -51.750168, -50.660168,
       -46.681023, -46.31015 , -45.479893, -43.51856 , -41.098

'id'

(12565,)

'XC483906.mp3'

In [337]:
display(val_y.shape)
display(val_y[:15])

(3318,)

array([ 3,  2,  1,  0,  0,  8, 19,  2, 19,  5, 19, 15, 11,  7,  2],
      dtype=int32)

In [338]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(3318, 128)

array([-0.01707026, -0.39656842, -0.4183283 , -0.45894244, -0.49384403,
       -0.51980877, -0.52839357, -0.5510067 , -0.58286625, -0.5928059 ,
       -0.6128816 , -0.61979336, -0.6288633 , -0.64968634, -0.6670257 ,
       -0.6761421 , -0.67730856, -0.6816009 , -0.67545867, -0.6647226 ,
       -0.6719049 , -0.6619738 , -0.6471278 , -0.652049  , -0.64146125,
       -0.64391303, -0.6518209 , -0.67124003, -0.71799076, -0.70881754,
       -0.7226164 , -0.73732513, -0.7245811 , -0.71309674, -0.7026587 ,
       -0.7268016 , -0.76876825, -0.75947165, -0.76661104, -0.7662666 ,
       -0.76978624, -0.7542365 , -0.7629837 , -0.7993014 , -0.8069032 ,
       -0.80255187, -0.80604035, -0.8206661 , -0.817362  , -0.81978494,
       -0.83180475, -0.81759435, -0.8207392 , -0.82689744, -0.83627135,
       -0.8303414 , -0.8460805 , -0.85512656, -0.8524952 , -0.84508985,
       -0.83099365, -0.8284293 , -0.8292149 , -0.83307105, -0.8172412 ,
       -0.82091004, -0.8238109 , -0.8224975 , -0.8299492 , -0.82

'mfcc'

(3318, 20)

...

(3318, 84)

array([-19.192596 , -18.357492 , -18.763315 , -19.366571 , -16.929043 ,
       -14.4525385, -14.049117 , -12.759987 , -14.402209 , -17.90481  ,
       -16.231672 , -15.364329 , -16.775997 , -17.590565 , -16.408525 ,
       -19.18176  , -19.740923 , -20.840792 , -21.245556 , -20.723444 ,
       -20.999773 , -25.08835  , -23.356092 , -25.212355 , -26.946945 ,
       -24.921827 , -25.974691 , -28.842213 , -28.154224 , -28.030458 ,
       -28.919643 , -29.377167 , -31.023067 , -31.180063 , -34.014633 ,
       -33.669846 , -34.84148  , -35.023224 , -36.85372  , -36.162834 ,
       -37.3165   , -37.896534 , -40.190365 , -40.017704 , -41.065468 ,
       -40.98337  , -41.63271  , -41.198174 , -40.882187 , -40.49316  ,
       -39.26483  , -39.1765   , -38.733685 , -39.23859  , -42.05581  ,
       -44.75164  , -46.034424 , -44.174923 , -45.603477 , -48.402534 ,
       -48.42529  , -48.000134 , -50.627438 , -51.769367 , -53.32704  ,
       -52.579865 , -53.69759  , -53.566704 , -54.655746 , -54.5

'id'

(3318,)

'XC428404.mp3'

## **Random Forest Model**

Saving evaluation results

In [339]:
def evaluate_model(model, validation_features, val_y):
    # Predict class labels for validation set
    val_yhat_result = model.predict(validation_features)

    # Print classification report
    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Get probabilities for the validation set (for AUC calculation)
    val_y_proba = model.predict_proba(validation_features)

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = roc_auc_score(val_y, val_y_proba, multi_class='ovr', average='weighted')
    print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    # Store the scores in the dictionary
    val_score = {'f1': f1, 'auc': auc_score}

    # Return the scores dictionary
    return val_score

In [340]:
train_results = {}
val_results = {}

val_scores = {}
k=15

### **With all the features**

In [341]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['cqt'], train_features['melspectrogram']), axis=1)

training_features.shape

(12565, 244)

In [342]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['cqt'], val_features['melspectrogram']), axis=1)

validation_features.shape

(3318, 244)

Fit the model with training data

In [343]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [344]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['all_features'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['all_features'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.595539481615431


In [345]:
val_scores['all_features'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.56      0.59      0.58       455
           1       0.43      0.65      0.52       492
           2       0.67      0.63      0.65       889
           3       0.93      0.95      0.94       150
           4       0.08      0.03      0.04        67
           5       0.46      0.91      0.61        43
           6       0.19      0.12      0.15        24
           7       0.98      0.93      0.95        44
           8       0.40      0.64      0.49        50
           9       0.94      0.70      0.80       169
          10       0.65      0.53      0.58        53
          11       0.30      0.26      0.28        66
          12       0.97      0.59      0.74        59
          13       0.60      0.53      0.56        57
          14       0.39      0.63      0.48        38
          15       0.41      0.28      0.33       311
          16       0.94      0.89      0.92   

### **With MFCCs only**

In [346]:
training_features = train_features['mfcc']

training_features.shape

(12565, 20)

In [347]:
validation_features = val_features['mfcc']

validation_features.shape

(3318, 20)

In [348]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [349]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5581675708257987


In [350]:
val_scores['mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.55      0.62      0.58       455
           1       0.32      0.57      0.41       492
           2       0.63      0.46      0.53       889
           3       0.81      1.00      0.90       150
           4       0.43      0.04      0.08        67
           5       0.62      0.56      0.59        43
           6       0.26      0.25      0.26        24
           7       1.00      0.93      0.96        44
           8       0.44      0.62      0.51        50
           9       0.92      0.86      0.89       169
          10       0.15      0.06      0.08        53
          11       0.38      0.27      0.32        66
          12       0.97      0.64      0.78        59
          13       0.61      0.61      0.61        57
          14       0.47      0.71      0.56        38
          15       0.55      0.50      0.52       311
          16       0.84      0.98      0.91   

### **With Mel-Spectrogram Only**

In [351]:
training_features = train_features['melspectrogram']

training_features.shape

(12565, 128)

In [352]:
validation_features = val_features['melspectrogram']

validation_features.shape

(3318, 128)

In [353]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [354]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5280289330922242


In [355]:
val_scores['melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.64      0.57      0.60       455
           1       0.36      0.59      0.44       492
           2       0.62      0.47      0.54       889
           3       0.92      0.89      0.91       150
           4       0.08      0.04      0.06        67
           5       0.28      0.72      0.41        43
           6       0.07      0.21      0.10        24
           7       0.84      0.61      0.71        44
           8       0.44      0.62      0.52        50
           9       0.93      0.60      0.73       169
          10       0.35      0.47      0.40        53
          11       0.21      0.14      0.17        66
          12       0.77      0.39      0.52        59
          13       0.62      0.35      0.45        57
          14       0.24      0.55      0.34        38
          15       0.44      0.40      0.42       311
          16       0.89      1.00      0.94   

## **With Chroma Only**

In [356]:
training_features = train_features['chroma']

training_features.shape

(12565, 12)

In [357]:
validation_features = val_features['chroma']

validation_features.shape

(3318, 12)

In [358]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [359]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3595539481615431


In [360]:
val_scores['chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.44      0.41      0.43       455
           1       0.25      0.49      0.33       492
           2       0.43      0.26      0.32       889
           3       0.41      0.61      0.49       150
           4       0.09      0.07      0.08        67
           5       0.08      0.02      0.04        43
           6       0.06      0.04      0.05        24
           7       0.44      0.57      0.50        44
           8       0.04      0.04      0.04        50
           9       0.81      0.64      0.71       169
          10       0.05      0.02      0.03        53
          11       0.14      0.20      0.17        66
          12       0.54      0.25      0.34        59
          13       0.10      0.11      0.10        57
          14       0.22      0.18      0.20        38
          15       0.47      0.45      0.46       311
          16       0.25      0.49      0.33   

## **With CQT Only**

In [361]:
training_features = train_features['cqt']

training_features.shape

(12565, 84)

In [362]:
validation_features = val_features['cqt']

validation_features.shape

(3318, 84)

In [363]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [364]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.4267631103074141


In [365]:
val_scores['cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.49      0.56      0.52       455
           1       0.30      0.47      0.37       492
           2       0.53      0.40      0.46       889
           3       0.33      0.47      0.39       150
           4       0.00      0.00      0.00        67
           5       0.53      0.67      0.59        43
           6       0.10      0.21      0.14        24
           7       0.95      0.82      0.88        44
           8       0.26      0.48      0.34        50
           9       0.76      0.63      0.69       169
          10       0.28      0.42      0.34        53
          11       0.21      0.23      0.22        66
          12       0.66      0.66      0.66        59
          13       0.42      0.30      0.35        57
          14       0.16      0.18      0.17        38
          15       0.28      0.25      0.26       311
          16       0.75      0.89      0.82   

## **With MFCCs and Mel-Spectrogram**

In [366]:
training_features = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)

training_features.shape

(12565, 148)

In [367]:
validation_features = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)

validation_features.shape

(3318, 148)

In [368]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [369]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.578963230861965


In [370]:
val_scores['mfcc_melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.63      0.61      0.62       455
           1       0.38      0.61      0.47       492
           2       0.63      0.52      0.57       889
           3       0.93      0.97      0.95       150
           4       0.07      0.03      0.04        67
           5       0.34      0.86      0.49        43
           6       0.19      0.25      0.21        24
           7       0.97      0.82      0.89        44
           8       0.44      0.64      0.52        50
           9       0.89      0.60      0.71       169
          10       0.55      0.34      0.42        53
          11       0.25      0.21      0.23        66
          12       0.95      0.66      0.78        59
          13       0.68      0.53      0.59        57
          14       0.42      0.63      0.51        38
          15       0.55      0.54      0.55       311
          16       0.90      0.98      0.94   

## **With MFCCs and Chroma**

In [371]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)

training_features.shape

(12565, 32)

In [372]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)

validation_features.shape

(3318, 32)

In [373]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [374]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5922242314647378


In [375]:
val_scores['mfcc_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.62      0.66      0.64       455
           1       0.34      0.61      0.44       492
           2       0.66      0.51      0.58       889
           3       0.85      1.00      0.92       150
           4       0.12      0.03      0.05        67
           5       0.82      0.65      0.73        43
           6       0.23      0.21      0.22        24
           7       0.96      0.98      0.97        44
           8       0.52      0.60      0.56        50
           9       0.91      0.80      0.86       169
          10       0.54      0.25      0.34        53
          11       0.57      0.32      0.41        66
          12       0.97      0.51      0.67        59
          13       0.58      0.58      0.58        57
          14       0.47      0.68      0.56        38
          15       0.63      0.57      0.60       311
          16       0.81      0.93      0.86   

## **With MFCCs and CQT**

In [376]:
training_features = np.concatenate((train_features['mfcc'], train_features['cqt']), axis=1)

training_features.shape

(12565, 104)

In [377]:
validation_features = np.concatenate((val_features['mfcc'], val_features['cqt']), axis=1)

validation_features.shape

(3318, 104)

In [378]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [379]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5385774562989752


In [380]:
val_scores['mfcc_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.63      0.60      0.61       455
           1       0.36      0.59      0.44       492
           2       0.59      0.53      0.56       889
           3       0.78      0.95      0.86       150
           4       0.17      0.03      0.05        67
           5       0.58      0.88      0.70        43
           6       0.19      0.21      0.20        24
           7       1.00      0.93      0.96        44
           8       0.36      0.60      0.45        50
           9       0.89      0.59      0.71       169
          10       0.25      0.36      0.30        53
          11       0.23      0.27      0.25        66
          12       0.85      0.66      0.74        59
          13       0.57      0.60      0.58        57
          14       0.52      0.66      0.58        38
          15       0.38      0.31      0.34       311
          16       0.84      0.87      0.86   

## **With Chroma and CQT**

In [381]:
training_features = np.concatenate((train_features['chroma'], train_features['cqt']), axis=1)

training_features.shape

(12565, 96)

In [382]:
validation_features = np.concatenate((val_features['chroma'], val_features['cqt']), axis=1)

validation_features.shape

(3318, 96)

In [383]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [384]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.49969861362266427


In [385]:
val_scores['chroma_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.57      0.60      0.59       455
           1       0.35      0.52      0.42       492
           2       0.58      0.53      0.55       889
           3       0.66      0.74      0.70       150
           4       0.00      0.00      0.00        67
           5       0.50      0.53      0.52        43
           6       0.14      0.17      0.15        24
           7       1.00      0.89      0.94        44
           8       0.27      0.46      0.34        50
           9       0.90      0.62      0.73       169
          10       0.33      0.62      0.43        53
          11       0.31      0.42      0.36        66
          12       0.78      0.53      0.63        59
          13       0.46      0.33      0.39        57
          14       0.21      0.24      0.23        38
          15       0.25      0.16      0.19       311
          16       0.75      0.78      0.77   

## **With Mel-Spectrogram and Chroma**

In [386]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)

training_features.shape

(12565, 140)

In [387]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)

validation_features.shape

(3318, 140)

In [388]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [389]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5482218203737191


In [390]:
val_scores['melspectrogram_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.70      0.63      0.66       455
           1       0.36      0.60      0.45       492
           2       0.61      0.47      0.53       889
           3       0.90      0.93      0.91       150
           4       0.14      0.09      0.11        67
           5       0.38      0.86      0.53        43
           6       0.13      0.25      0.17        24
           7       0.82      0.73      0.77        44
           8       0.38      0.60      0.47        50
           9       0.87      0.62      0.72       169
          10       0.57      0.60      0.59        53
          11       0.38      0.29      0.33        66
          12       0.80      0.27      0.41        59
          13       0.53      0.28      0.37        57
          14       0.28      0.55      0.38        38
          15       0.43      0.39      0.41       311
          16       0.71      0.96      0.82   

## **With Mel-Spectrogram and CQT**

In [391]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt']), axis=1)

training_features.shape

(12565, 212)

In [392]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt']), axis=1)

validation_features.shape

(3318, 212)

In [393]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [394]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5515370705244123


In [395]:
val_scores['melspectrogram_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.50      0.54      0.52       455
           1       0.44      0.63      0.52       492
           2       0.68      0.61      0.64       889
           3       0.89      0.84      0.86       150
           4       0.00      0.00      0.00        67
           5       0.44      0.81      0.57        43
           6       0.08      0.21      0.12        24
           7       0.89      0.91      0.90        44
           8       0.39      0.50      0.44        50
           9       0.92      0.69      0.79       169
          10       0.38      0.45      0.41        53
          11       0.21      0.17      0.19        66
          12       0.97      0.51      0.67        59
          13       0.42      0.30      0.35        57
          14       0.16      0.26      0.20        38
          15       0.38      0.28      0.32       311
          16       0.98      0.85      0.91   

## **With Mel-Spectrogram, MFCCs, CQT**

In [396]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt'], train_features['mfcc']), axis=1)

training_features.shape

(12565, 232)

In [397]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt'], val_features['mfcc']), axis=1)

validation_features.shape

(3318, 232)

In [398]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [399]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.581374321880651


In [400]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.52      0.55      0.54       455
           1       0.43      0.63      0.51       492
           2       0.68      0.63      0.66       889
           3       0.90      0.96      0.93       150
           4       0.07      0.03      0.04        67
           5       0.40      0.86      0.55        43
           6       0.10      0.12      0.11        24
           7       1.00      0.93      0.96        44
           8       0.46      0.62      0.53        50
           9       0.90      0.70      0.79       169
          10       0.55      0.45      0.49        53
          11       0.25      0.21      0.23        66
          12       0.95      0.63      0.76        59
          13       0.59      0.51      0.55        57
          14       0.39      0.58      0.47        38
          15       0.41      0.30      0.34       311
          16       0.96      0.91      0.93   

## **With Mel-Spectrogram, MFCCs, Chroma**

In [401]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)

training_features.shape

(12565, 160)

In [402]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)

validation_features.shape

(3318, 160)

In [403]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [404]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5925256178420736


In [405]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.68      0.65      0.66       455
           1       0.38      0.63      0.48       492
           2       0.64      0.53      0.58       889
           3       0.92      0.97      0.94       150
           4       0.10      0.04      0.06        67
           5       0.37      0.86      0.52        43
           6       0.27      0.25      0.26        24
           7       0.97      0.89      0.93        44
           8       0.38      0.60      0.47        50
           9       0.93      0.59      0.72       169
          10       0.68      0.53      0.60        53
          11       0.29      0.21      0.25        66
          12       1.00      0.51      0.67        59
          13       0.59      0.53      0.56        57
          14       0.41      0.63      0.49        38
          15       0.54      0.52      0.53       311
          16       0.83      0.96      0.89   

# Review results from all models

In [406]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.0,0.596
3,chroma,1.0,0.360
8,chroma_cqt,1.0,0.500
4,cqt,1.0,0.427
2,melspectrogram,1.0,0.528
9,melspectrogram_chroma,1.0,0.548
12,melspectrogram_chroma_mfcc,1.0,0.593
10,melspectrogram_cqt,1.0,0.552
11,melspectrogram_cqt_mfcc,1.0,0.581
1,mfcc,1.0,0.558


In [407]:
val_scores_df = pd.DataFrame([(key, value['f1'], value['auc']) for key, value in val_scores.items()],
                             columns=['Features', 'F1_Score', 'AUC_Score']).round(3)

val_scores_df = val_scores_df.sort_values('Features')
print(val_scores_df)

                   Features  F1_Score  AUC_Score
0              all_features     0.594      0.866
3                    chroma     0.356      0.747
8                chroma_cqt     0.497      0.826
4                       cqt     0.426      0.793
2            melspectrogram     0.537      0.834
9     melspectrogram_chroma     0.554      0.854
10       melspectrogram_cqt     0.554      0.849
11  melspectrogram_cqt_mfcc     0.597      0.871
1                      mfcc     0.557      0.871
6               mfcc_chroma     0.593      0.890
7                  mfcc_cqt     0.538      0.834
5       mfcc_melspectrogram     0.584      0.857
